# Covid-19 Data Analysis for visualize and understanding pandemic cases



In [ ]:
pip install plotly folium

In [ ]:
#required packages and libraries

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

import folium

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

import math
import random
from datetime import timedelta

import warnings
warnings.filterwarnings('ignore')

# color pallette
cnf ='#393e46'
dth ='#ff2e63'
rec ='#21bf73'
act ='#fe9801'


## Dataset Preparation

In [ ]:
import plotly as py
py.offline.init_notebook_mode(connected = True)

In [ ]:
import os

In [ ]:
# Understanding the Data
df = pd.read_csv('Covid-19-Preprocessed-Dataset/preprocessed/covid_19_data_cleaned.csv', parse_dates=['Date'])
df

In [ ]:
df['Province/State'] = df['Province/State'].fillna("")
df

In [ ]:
country_daywise = pd.read_csv('Covid-19-Preprocessed-Dataset/preprocessed/country_daywise.csv', parse_dates=['Date'])
countrywise = pd.read_csv('Covid-19-Preprocessed-Dataset/preprocessed/countrywise.csv')
daywise = pd.read_csv('Covid-19-Preprocessed-Dataset/preprocessed/daywise.csv', parse_dates=['Date'])

In [ ]:
country_daywise

## Worldwide Total Confirmed, Recovered Cases and Deaths

In [ ]:
confirmed = df.groupby('Date').sum()['Confirmed'].reset_index()
confirmed

In [ ]:
recovered = df.groupby('Date').sum()['Recovered'].reset_index()
recovered

In [ ]:
deaths = df.groupby('Date').sum()['Deaths'].reset_index()
deaths

In [ ]:
df.isnull().sum()

In [ ]:
df.query('Country == "US"')

## Scatterplot Visualization

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = confirmed['Date'], y= confirmed['Confirmed'], mode='lines+markers', name='Confirmed Cases', line=dict(color='Orange',width =2)))
fig.add_trace(go.Scatter(x = recovered['Date'], y= recovered['Recovered'], mode='lines+markers', name='Recovered Cases', line=dict(color='Green', width =2)))
fig.add_trace(go.Scatter(x = deaths['Date'], y= deaths['Deaths'], mode='lines+markers', name='Deaths', line=dict(color='Red', width=2)))

fig.update_layout(title = "WorldWide Covid-19 Cases", xaxis_tickfont_size=14, yaxis= dict(title='Number of Cases'))
fig.show()

## Cases Density Animation On World Map

In [ ]:
df['Date'] = df['Date'].astype(str)

In [ ]:
df.info()

In [ ]:
fig =px.density_mapbox(df, lat='Lat', lon='Long', hover_name='Country', hover_data=['Confirmed', 'Recovered', 'Deaths'], animation_frame='Date',color_continuous_scale='Portland', radius=7, zoom=0,height=700)
fig.update_layout(title='WorldWide Covid-19 Cases with TimeLapse')
fig.update_layout(mapbox_style='open-street-map', mapbox_center_lon=0)
fig.show()

## Cases over a time period with Area Plot

In [ ]:
df['Date'] =pd.to_datetime(df['Date'])
df.info()

## Cases visualization using Folium Maps

In [ ]:
temp =df[df['Date'] == max(df['Date'])]

m =folium.Map(location =[0,0], tiles='cartodbpositron', min_zoom =1, max_zoom= 4, zoom_start=1)

for i in range(0, len(temp)):
    folium.Circle(location = [temp.iloc[i]['Lat'], temp.iloc[i]['Long']], color='crimson', fill =  'crimson', 
                tooltip = '<li><bold> Country: ' + str(temp.iloc[i]['Country'])+
                          '<li><bold> Province: ' + str(temp.iloc[i]['Province/State'])+
                          '<li><bold> Confirmed: ' + str(temp.iloc[i]['Confirmed'])+
                          '<li><bold> Deaths: ' + str(temp.iloc[i]['Deaths']),
                radius = int(temp.iloc[i]['Confirmed'])**0.5).add_to(m)

m

## Confirmed Cases with Choropleth Map

In [ ]:
fig = px.choropleth(country_daywise, locations='Country', locationmode='country names', color=country_daywise['Confirmed'],
                   hover_name = 'Country', animation_frame=country_daywise['Date'].dt.strftime('%Y-%m-%d'),
                   title = 'Cases Over Time in Different Countries', color_continuous_scale=px.colors.sequential.Inferno)

fig.update(layout_coloraxis_showscale=True)
fig.show()

## Confirmed & Death Cases using Bar plots

In [ ]:
fig_c =px.bar(daywise, x='Date', y='Confirmed', color_discrete_sequence=[act])
fig_d =px.bar(daywise, x='Date', y='Deaths', color_discrete_sequence=[dth])

fig = make_subplots(rows =1, cols =2, shared_xaxes=False, horizontal_spacing= 0.1,
                   subplot_titles=('Confirmed Cases', 'Death Cases'))

fig.add_trace(fig_c['data'][0], row=1, col=1)
fig.add_trace(fig_d['data'][0], row=1, col=2)

fig.update_layout(height=400)
fig.show()


## Confirmed & Death Cases with Static Colormap

In [ ]:
fig_c = px.choropleth(countrywise, locations='Country', locationmode='country names',
                     color=np.log(countrywise['Confirmed']), hover_name='Country', hover_data=['Confirmed'])

temp = countrywise[countrywise['Deaths']> 0]
fig_d = px.choropleth(temp, locations='Country', locationmode='country names',
                     color=np.log(temp['Deaths']), hover_name='Country', hover_data=['Deaths'])

fig = make_subplots(rows=1, cols=2, subplot_titles=['Confirmed Cases', 'Deaths'], 
                   specs=[[{'type':'choropleth'},{'type':'choropleth'}]])

fig.add_trace(fig_c['data'][0], row=1, col=1)
fig.add_trace(fig_d['data'][0], row=1, col=2)

fig.update(layout_coloraxis_showscale=False)

fig.show()

## Deaths & Recoveries per 100 cases

 The Data is only accurate untill July 2021, after that time the data for recovery cases were not reported also was not being tracked by the institutes.

In [ ]:
daywise.columns

In [ ]:
fig1 = px.line(daywise, x='Date', y='Deaths / 100 Cases', color_discrete_sequence=[dth])
fig2 = px.line(daywise, x='Date', y='Recovered / 100 Cases', color_discrete_sequence=[rec])
fig3 = px.line(daywise, x='Date', y='Deaths / 100 Recovered', color_discrete_sequence=[rec])

fig = make_subplots(rows =1, cols=3, shared_xaxes=False, 
                   subplot_titles=('Deaths / 100 Cases','Recovered / 100 Cases','Deaths / 100 Recovered'))

fig.add_trace(fig1['data'][0], row=1,col=1)
fig.add_trace(fig2['data'][0], row=1,col=2)
fig.add_trace(fig3['data'][0], row=1,col=3)

fig.update_layout(height=400)
fig.show()

## Number of new cases per Day & Countries Affected

In [ ]:
fig_c = px.bar(daywise, x='Date', y='Confirmed', color_discrete_sequence=[act])
fig_d = px.bar(daywise, x='Date', y='No. of Countries', color_discrete_sequence=[dth])

fig =make_subplots(rows =1, cols=2, shared_xaxes=False, horizontal_spacing=0.1, subplot_titles=('No. of New Cases per Day', 'No. of Countries'))

fig.add_trace(fig_c['data'][0], row=1, col=1)
fig.add_trace(fig_d['data'][0], row=1, col=2)

fig.show()

## Top 15 Countries Case Analysis

In [ ]:
countrywise.columns

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
top =15

# confirmed cases
fig_c = px.bar(countrywise.sort_values('Confirmed').tail(top), x='Confirmed', y='Country',
              text = 'Confirmed', orientation='h', color_discrete_sequence=[cnf])

# death cases
fig_d = px.bar(countrywise.sort_values('Deaths').tail(top), x='Deaths', y='Country',
              text = 'Deaths', orientation='h', color_discrete_sequence=[dth])

# active cases
fig_a = px.bar(countrywise.sort_values('Active').tail(top), x='Active', y='Country',
              text = 'Active', orientation='h', color_discrete_sequence=[act])

# deaths / 100 cases
fig_dc = px.bar(countrywise.sort_values('Deaths / 100 Cases').tail(top), x='Deaths / 100 Cases', y='Country',
              text = 'Deaths / 100 Cases', orientation='h', color_discrete_sequence=['#ff0000'])


# new cases country wise
fig_nc = px.bar(countrywise.sort_values('New Cases').tail(top), x='New Cases', y='Country',
              text = 'New Cases', orientation='h', color_discrete_sequence=['#944dff'])

temp= countrywise[countrywise['Population']>1000000]
# cases per million people
fig_p = px.bar(temp.sort_values('Cases / Million People').tail(top), x='Cases / Million People', y='Country',
              text = 'Cases / Million People', orientation='h', color_discrete_sequence=['#3366ff'])

# 1 week changes
fig_ow = px.bar(countrywise.sort_values('1 week change').tail(top), x='1 week change', y='Country',
              text = '1 week change', orientation='h', color_discrete_sequence=['#ff6600'])

# 1 week % increase
tem= countrywise[countrywise['Confirmed']>100]
fig_op = px.bar(tem.sort_values('1 week % increase').tail(top), x='1 week % increase', y='Country',
              text = '1 week % increase', orientation='h', color_discrete_sequence=['#990033'])



fig= make_subplots(rows=4 , cols=2, shared_xaxes=False, horizontal_spacing= 0.2, vertical_spacing=.05,
                  subplot_titles=('Confirmed Cases', 'Deaths Reported', 'Active Cases', 'Deaths / 100 Cases'
                                 ,'New Cases', 'Cases / Million People', '1 week change', '1 week % increase'))

fig.add_trace(fig_c['data'][0], row=1, col=1)
fig.add_trace(fig_d['data'][0], row=1, col=2)

fig.add_trace(fig_a['data'][0], row=2, col=1)
fig.add_trace(fig_dc['data'][0], row=2, col=2)


fig.add_trace(fig_nc['data'][0], row=3, col=1)
fig.add_trace(fig_p['data'][0], row=3, col=2)


fig.add_trace(fig_ow['data'][0], row=4, col=1)
fig.add_trace(fig_op['data'][0], row=4, col=2)


fig.update_layout(height=3000)
fig.show()

## Scatter Plot for Deaths VS Confirmed Cases

In [ ]:
countrywise.sort_values('Deaths', ascending=False).head(15)

In [ ]:
top =15
fig = px.scatter(countrywise.sort_values('Deaths', ascending=False).head(top), 
                x = 'Confirmed', y='Deaths', color='Country', height=600, size='Confirmed',
                text='Country', log_x = True, log_y = True, title='Deaths vs Confirmed Cases(Cases are on log10 scale)')

fig.update_traces(textposition= 'top center')
fig.update_layout(showlegend = True)
fig.update_layout(xaxis_rangeslider_visible = True)
fig.show()

## Stacked Bar Plot

In [ ]:
fig = px.bar(country_daywise, x='Date', y='Confirmed', color='Country', height=600,
            title='Confirmed Cases', color_discrete_sequence=px.colors.cyclical.mygbm)
fig.show()

In [ ]:
fig = px.bar(country_daywise, x='Date', y='Deaths', color='Country', height=600,
            title='Deaths', color_discrete_sequence=px.colors.cyclical.mygbm)
fig.show()

In [ ]:
fig = px.bar(country_daywise, x='Date', y='New Cases', color='Country', height=600,
            title='New Cases', color_discrete_sequence=px.colors.cyclical.mygbm)
fig.show()

## Line Plot

In [ ]:
fig = px.line(country_daywise, x ='Date', y='Confirmed', color='Country', height=600, title='Confirmed Cases',
             color_discrete_sequence=px.colors.cyclical.mygbm)
fig.show()

In [ ]:
fig = px.line(country_daywise, x ='Date', y='Deaths', color='Country', height=600, title='Deaths',
             color_discrete_sequence=px.colors.cyclical.mygbm)
fig.show()

In [ ]:
fig = px.line(country_daywise, x ='Date', y='New Cases', color='Country', height=600, title='New Cases',
             color_discrete_sequence=px.colors.cyclical.mygbm)
fig.show()

### Growth rate after 100K Cases

In [ ]:
gt_100000 = country_daywise[country_daywise['Confirmed']>100000]['Country'].unique()
temp = df[df['Country'].isin(gt_100000)]

temp = temp.groupby(['Country', 'Date'])['Confirmed'].sum().reset_index()
temp =temp[temp['Confirmed']>100000]

min_date = temp.groupby('Country')['Date'].min().reset_index()
min_date.columns = ['Country', 'Min Date']

from_100000th_case = pd.merge(temp, min_date,on ='Country')
from_100000th_case['N days'] = (from_100000th_case['Date'] - from_100000th_case['Min Date']).dt.days

fig = px.line(from_100000th_case, x = 'N days', y='Confirmed', color='Country', title='N Days From 100K Cases', height=600)
fig.show()

### Growth rate after 1Million Cases

In [ ]:
gt_1000000 = country_daywise[country_daywise['Confirmed']>1000000]['Country'].unique()
temp = df[df['Country'].isin(gt_1000000)]

temp = temp.groupby(['Country', 'Date'])['Confirmed'].sum().reset_index()
temp =temp[temp['Confirmed']>1000000]

min_date = temp.groupby('Country')['Date'].min().reset_index()
min_date.columns = ['Country', 'Min Date']

from_1000000th_case = pd.merge(temp, min_date,on ='Country')
from_1000000th_case['N days'] = (from_1000000th_case['Date'] - from_1000000th_case['Min Date']).dt.days

fig = px.line(from_1000000th_case, x = 'N days', y='Confirmed', color='Country', title='N Days From 1Million Cases', height=600)
fig.show()

### Growth rate after 10Million Cases

In [ ]:
gt_10000000 = country_daywise[country_daywise['Confirmed']>10000000]['Country'].unique()
temp = df[df['Country'].isin(gt_10000000)]

temp = temp.groupby(['Country', 'Date'])['Confirmed'].sum().reset_index()
temp =temp[temp['Confirmed']>10000000]

min_date = temp.groupby('Country')['Date'].min().reset_index()
min_date.columns = ['Country', 'Min Date']

from_10000000th_case = pd.merge(temp, min_date,on ='Country')
from_10000000th_case['N days'] = (from_10000000th_case['Date'] - from_10000000th_case['Min Date']).dt.days

fig = px.line(from_10000000th_case, x = 'N days', y='Confirmed', color='Country', title='N Days From 10Million Cases', height=600)
fig.show()

## Tree Map Analysis

In [ ]:
full_latest = df[df['Date'] == max(df['Date'])]

fig = px.treemap(full_latest.sort_values(by='Confirmed', ascending=False).reset_index(drop=True),
                path = ['Country', 'Province/State'], values='Confirmed', height=700,
                title='Number of Confirmed Cases',
                color_discrete_sequence=px.colors.qualitative.Dark2)

fig.data[0].textinfo= 'label+text+value'
fig.show()

In [ ]:
full_latest = df[df['Date'] == max(df['Date'])]

fig = px.treemap(full_latest.sort_values(by='Deaths', ascending=False).reset_index(drop=True),
                path = ['Country', 'Province/State'], values='Deaths', height=700,
                title='Number of Deaths',
                color_discrete_sequence=px.colors.qualitative.Dark2)

fig.data[0].textinfo= 'label+text+value'
fig.show()

## Confirmed Cases Country & Day Wise

In [ ]:
temp = country_daywise.groupby(['Country', 'Date'])['Confirmed'].sum().reset_index()
temp = temp[temp['Country'].isin(gt_10000000)]

countries= temp['Country'].unique()


ncols = 3
nrows = math.ceil(len(countries)/ncols)

fig = make_subplots(rows = nrows, cols= ncols, shared_xaxes= False, subplot_titles= countries)

for ind, country in enumerate(countries):
    row = int((ind/ncols)+1)
    col = int((ind%ncols)+1)
    fig.add_trace(go.Bar(x= temp['Date'], y=temp.loc[temp['Country']== country, 'Confirmed'], name=country), row= row, col=col)
    
fig.update_layout(height=4000, title_text='Confirmed Cases in each Country')    
fig.update_layout(showlegend=False)
fig.show()


## Covid-19 VS Other Similar Epidemics

In [ ]:
# Source for the number wikipedia
epidemics = pd.DataFrame({
    'epidemic' :['COVID-19', 'SARS', 'EBOLA', 'MERS', 'H1N1'],
    'start_year' : [2019, 2002, 2013, 2012, 2009],
    'end_year' :[2020, 2004, 2016, 2020, 2010],
    'confirmed' :[full_latest['Confirmed'].sum(), 8422, 28646, 2519, 6724149],
    'deaths': [full_latest['Deaths'].sum(), 813, 11323, 866, 19654]
})


In [ ]:
# Calculating mortality rate
epidemics['mortality'] =round((epidemics['deaths']/epidemics['confirmed'])*100, 2)

temp = epidemics.melt(id_vars='epidemic', value_vars=['confirmed', 'deaths', 'mortality'], 
                     var_name='Case', value_name='Value')

fig= px.bar(temp, x='epidemic', y='Value', color= 'epidemic', text='Value', facet_col='Case',
           color_discrete_sequence=px.colors.qualitative.Bold)

fig.update_traces(textposition='outside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode= 'hide')
fig.update_yaxes(showticklabels= False)
fig.layout.yaxis2.update(matches = None)
fig.layout.yaxis3.update(matches = None)
fig.show()